In [0]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), '..'))
	print(os.getcwd())
except:
	pass


 # Demo 4: SQL Server コンテナ内のDBバックアップ
 ## 4-1.フルバックアップ

In [20]:
%%bash
sqlcmd -S localhost,11433 -U sa -P $PASSWORD -Q 'backup database DevDB to disk="/tmp/DevDB.bak" with format'
sudo docker exec sql19rc1 ls -la /tmp/DevDB.bak


Processed 384 pages for database 'DevDB', file 'DevDB' on file 1.
Processed 2 pages for database 'DevDB', file 'DevDB_log' on file 1.
BACKUP DATABASE successfully processed 386 pages in 0.605 seconds (4.978 MB/sec).
-rw-r----- 1 root root 3264512 Oct 26 04:19 /tmp/DevDB.bak


 ## 4-2.バックアップをホストにコピー

In [21]:
%%bash
sudo docker cp sql19rc1:/tmp/DevDB.bak ~/SQL_Server_Container/DevDB.bak
ls -la ~/SQL_Server_Container/DevDB.bak


-rw-r----- 1 root root 3264512 Oct 26 04:19 /home/o9o9/SQL_Server_Container/DevDB.bak


 ## 4-3.DB削除＆DBファイル削除

In [22]:
%%bash
sudo docker rm -f sql19rc1
sudo docker ps -a --filter name=sql19rc1
sudo rm -r ~/SQL_Server_Container/DataVolume/*


sql19rc1
CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


 # Demo 5: SQL Server カスタムコンテナイメージの作成と配布
 ## 5-1.Dockerfile編集

In [23]:
%%bash
cd ~/SQL_Server_Container
touch Dockerfile
echo 'FROM mcr.microsoft.com/mssql/server:2019-RC1' >> Dockerfile
echo 'COPY ./DevDB.bak /tmp/DevDB.bak' >> Dockerfile
echo 'CMD ["/opt/mssql/bin/sqlservr"]' >> Dockerfile
cat Dockerfile


FROM mcr.microsoft.com/mssql/server:2019-RC1
COPY ./DevDB.bak /tmp/DevDB.bak
CMD ["/opt/mssql/bin/sqlservr"]


 ## 5-2.カスタムコンテナイメージのビルド(数十秒かかる)

In [24]:
%%bash
cd ~/SQL_Server_Container
sudo docker build -t localhost:5000/mssql/devdb:v1 .
sudo docker history localhost:5000/mssql/devdb:v1
sudo docker image list | grep devdb



Step 1/3 : FROM mcr.microsoft.com/mssql/server:2019-RC1
 ---> c5a295efea97
Step 2/3 : COPY ./DevDB.bak /tmp/DevDB.bak
 ---> ba76485b7cd9
Step 3/3 : CMD ["/opt/mssql/bin/sqlservr"]
 ---> Running in 99308259f4ae
Removing intermediate container 99308259f4ae
 ---> 496f00011d41
Successfully built 496f00011d41
Successfully tagged localhost:5000/mssql/devdb:v1
IMAGE               CREATED                  CREATED BY                                      SIZE                COMMENT
496f00011d41        Less than a second ago   /bin/sh -c #(nop)  CMD ["/opt/mssql/bin/sqls…   0B                  
ba76485b7cd9        37 seconds ago           /bin/sh -c #(nop) COPY file:7f070b8003992810…   3.26MB              
c5a295efea97        2 months ago             /bin/sh -c #(nop)  CMD ["/opt/mssql/bin/sqls…   0B                  
<missing>           2 months ago             /bin/sh -c #(nop)  ENTRYPOINT ["/opt/mssql/b…   0B                  
<missing>           2 months ago             /bin/sh -c /tmp/insta

 ## 5-3.ローカルプライベートレジストリ起動

In [25]:
%%bash
sudo docker run -d -p 5000:5000 --name registry -v ~/SQL_Server_Container/RegistryVolume:/var/lib/registry registry


0a90a64be05751f551e3e4a1e90d1f93a48d9acb6d3295e8d73df67207b44ecd


 ## 5-4.ローカルプライベートレジストリにプッシュ（事前プッシュ済みなので早い）

In [26]:
%%bash
sudo docker push localhost:5000/mssql/devdb:v1


The push refers to repository [localhost:5000/mssql/devdb]
55f61fa00fe9: Preparing
291d0eb6856c: Preparing
64a2fdccc647: Preparing
25ed665d515e: Preparing
e4e0ec1b4536: Preparing
2de391e51d73: Preparing
d73dd9e65295: Preparing
686245e78935: Preparing
d7ff1dc646ba: Preparing
644879075e24: Preparing
d73dd9e65295: Waiting
686245e78935: Waiting
d7ff1dc646ba: Waiting
644879075e24: Waiting
2de391e51d73: Waiting
291d0eb6856c: Layer already exists
64a2fdccc647: Layer already exists
e4e0ec1b4536: Layer already exists
25ed665d515e: Layer already exists
d73dd9e65295: Layer already exists
2de391e51d73: Layer already exists
686245e78935: Layer already exists
d7ff1dc646ba: Layer already exists
644879075e24: Layer already exists
55f61fa00fe9: Pushed
v1: digest: sha256:0876e786cd8d1d1008d7163627a1dfd01a3b57fc75de738e1683000f9e0a1421 size: 2411


 ## 5-5.ローカルプライベートレジストリ確認

In [27]:
%%bash
wget http://localhost:5000/v2/_catalog -q -O /dev/stdout
wget http://localhost:5000/v2/mssql/devdb/tags/list -q -O /dev/stdout


{"repositories":["mssql/devdb"]}
{"name":"mssql/devdb","tags":["v1"]}


 ## 5-6.ローカルに作成したカスタムコンテナイメージを削除（明示的にプライベートレジストリからPULLするため）

In [28]:
%%bash
sudo docker rmi localhost:5000/mssql/devdb:v1
sudo docker image list


Untagged: localhost:5000/mssql/devdb:v1
Untagged: localhost:5000/mssql/devdb@sha256:0876e786cd8d1d1008d7163627a1dfd01a3b57fc75de738e1683000f9e0a1421
Deleted: sha256:496f00011d417fc9a616a5bedf7f3680c8dc89067898eaefc2558ce25f316771
Deleted: sha256:ba76485b7cd9a7082b97ddfa0cc640c53d840587205690781b73b4f7ea671c48
Deleted: sha256:35f4a6d454018efe68a5f7ebb12e8a3c4bb7a44cf67d7d2082847a3ef666c914
REPOSITORY                       TAG                 IMAGE ID            CREATED             SIZE
mcr.microsoft.com/mssql/server   2019-RC1            c5a295efea97        2 months ago        1.5GB
mcr.microsoft.com/mssql/server   2019-latest         c5a295efea97        2 months ago        1.5GB
mcr.microsoft.com/mssql/server   2017-CU16           d9b9b96627b7        2 months ago        1.36GB
registry                         latest              f32a97de94e1        7 months ago        25.8MB


 ## 5-7.ローカルプライベートレジストリから Pull & Run

In [29]:
%%bash
sudo docker run -e ACCEPT_EULA=Y -e MSSQL_SA_PASSWORD=$PASSWORD -p 11433:1433 -v ~/SQL_Server_Container/DataVolume:/var/opt/mssql --name sql19rc1 -d localhost:5000/mssql/devdb:v1
sudo docker ps -a --filter name=sql19rc1
sudo docker image list | grep devdb


a44349de0fd81403b31e10bac2d5ee6f41a3379d1d5fa344839a472876e9f3c5
CONTAINER ID        IMAGE                           COMMAND                  CREATED             STATUS                  PORTS                     NAMES
a44349de0fd8        localhost:5000/mssql/devdb:v1   "/opt/mssql/bin/perm…"   2 seconds ago       Up Less than a second   0.0.0.0:11433->1433/tcp   sql19rc1
localhost:5000/mssql/devdb       v1                  496f00011d41        59 seconds ago      1.5GB
Unable to find image 'localhost:5000/mssql/devdb:v1' locally
v1: Pulling from mssql/devdb
b234f539f7a1: Already exists
55172d420b43: Already exists
5ba5bbeb6b91: Already exists
43ae2841ad7a: Already exists
f6c9c6de4190: Already exists
e39f945bda21: Already exists
6689ce95f395: Already exists
ec004dcfdfb5: Already exists
e44708601d04: Already exists
c6d51acc712c: Pulling fs layer
c6d51acc712c: Verifying Checksum
c6d51acc712c: Download complete
c6d51acc712c: Pull complete
Digest: sha256:0876e786cd8d1d1008d7163627a1dfd01a3b5

 ## 5-8.カスタム（バックアップファイルを配置）を確認

In [30]:
%%bash
sudo docker exec sql19rc1 ls -l /tmp


total 3188
-rw-r----- 1 root root 3264512 Oct 26 04:19 DevDB.bak
